# [Ria](https://ria.ru/) parse

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import re
from datetime import date
from dateutil.rrule import rrule, DAILY
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException


In [3]:
start_date = date(2019, 1, 1)
end_date = date(2023, 12, 31)
dates = []
for d in rrule(DAILY, dtstart=start_date, until=end_date):
    dates.append('https://ria.ru/economy/' + d.strftime("%Y%m%d"))

In [6]:
import math

urls = []
for n in tqdm(dates):
    url = n
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    materials = int(
        soup.find_all(
            'div', {"class": "rubric-count m-active"})[0].text.split(' ')[0])
    pages = math.ceil(materials / 20)
    date = url[-8:]
    for i in range(1, pages + 1):
        page = requests.get(url + '?page={}'.format(i))
        soup = BeautifulSoup(page.text, 'html')
        urls.extend([
            link.get('href') for link in soup.find_all(
                'a', {"class": "list-item__title color-font-hover-only"})
            if date in link.get('href')
        ])

In [57]:
def get_text(driver):
    paragraphs = driver.find_elements(By.CLASS_NAME, "article__text")
    full_text = '\n'.join([p.text for p in paragraphs])
    return full_text

In [58]:
def get_tags(driver):
    try:
        tags = driver.find_elements(By.CLASS_NAME, "article__tags-item")
        return [t.text for t in tags]
    except NoSuchElementException:
        return None

In [59]:
def get_time(driver):
    time = driver.find_element(By.CLASS_NAME, "article__info-date")
    return time.text

In [65]:
def get_header(driver):
    header = driver.find_element(By.CLASS_NAME, "article__title")
    return header.text

In [69]:
def get_category(driver):
    try:
        h1 = driver.find_element(By.CLASS_NAME, 'article__supertag-header-title')
        return h1.text
    except NoSuchElementException:
        return "Экономика"

In [70]:
def get_line(url0, driver):
    driver.get(url0)
    website = 'РИА'
    section = get_category(driver)
    header = get_header(driver)
    body = get_text(driver)
    date = get_time(driver)
    tags = get_tags(driver)
    return website, section, url0, header, body, date, tags

In [ ]:
dataset = []
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrome_options)
for url in tqdm(urls):
    dataset.append(get_line(url, driver))
driver.close()

In [74]:
df = pd.DataFrame(dataset)
df.columns = ['website', 'section', 'url', 'header', 'body', 'date', 'tags']

In [76]:
df.to_parquet('ria.parquet')